#  EPA-122A *Spatial* Data Science

## Lab 3 Part 1:  Exploratory Data Analysis, `seaborn`


**TU Delft**<br>
**Q1 2022**<br>
**Instructors:** Giacomo Marangoni, Theodoros Chatzivasileiadis <br>
**TAs:** Auriane Técourt, Dorukhan Yeşilli, Ludovica Bindi, Nicolò Canal, Ruth Nelson, Vaibhavi Srivastava <br>

---

## Table of Contents

* [Learning Goals](#chapter0)
* [Visualization Inspiration](#chapter1)
* [Plotting using `matplotlib`  and `seaborn`.](#chapter2)
    * [Introduction to `seaborn`](#section2_1)
* [Different Plots](#chapter3)
    * [Histograms](#section3_1)
    * [Kernel Density Estimation](#section3_2)
    * [Boxplots](#section3_3)
    * [Scatterplots](#section3_4)
    * [Pairplots](#section3_5)
    * [Plotting categorical variables](#section3_6)
* [Introduction to pandas plotting](#chapter4)
    * [Line graph](#section4_1)
    * [Bar plots](#section4_2)
    * [Histograms](#section4_3)
    * [Boxplots](#section4_4)
    * [Area plots](#section4_5)
    * [Scatterplots](#section4_6)
* [Excercise: EDA on Medical Data](#chapter5)

In [ ]:
# import the necessary libraries
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import requests
import IPython
from IPython.core.display import HTML

pd.set_option("display.width", 500)
pd.set_option("display.max_columns", 200)
pd.set_option("display.notebook_repr_html", True)
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import warnings

warnings.filterwarnings("ignore")
%config InlineBackend.figure_format ='retina'

## Learning Goals <a class="anchor" id="chapter0"></a>

By the end of this lab, you should be able to:
- Implement the different types of plots such as **histograms, boxplots** etc, that were mentioned in class.
- Develop an understanding of how to use `seaborn` and `matplotlib` and have both in your plotting toolbox.


## Visualization Inspiration <a class="anchor" id="chapter1"></a>

![title](figs/WGOITG.jpg)

[source: nytimes.org](https://www.nytimes.com/2018/08/27/learning/whats-going-on-in-this-graph-is-now-weekly-stem-teachers-explain-why-its-a-powerful-activity.html)

Notice that in “Summers Are Getting Hotter,” above, the histogram has intervals for global summer temperatures on the x-axis, designated from extremely cold to extremely hot, and their frequency on the y-axis.

That was an **infographic** intended for the general public. In contrast, take a look at the plots below of the same type of data published at a **scientific journal**. They look quite different, don't they?

![title](figs/F4.jpg)

*James Hansen, Makiko Sato, and Reto Ruedy*, Perception of climate change. [PNAS](https://www.pnas.org/content/109/37/E2415.full#abstract-1)

## Plotting using `matplotlib`  and `seaborn`. <a class="anchor" id="chapter2"></a>

Before you start coding your visualization, you need to decide what **type** of vizualization to use. A box plot, a histogram, a scatter plot, or something else? That will depend on the purpose of the plot (is it for performing an inspection on your data (EDA, or for showing your results/conclusions to people) and the number variables that you want to plot.

You have a lot of tools for plotting in Python. The basic one, of course, is `matplotlib` and there are other libraries that are built on top of it, such as `seaborn`, `bokeh`, or `altair`.

In this class we will continue using `matplotlib` and also look into [`seaborn`](https://seaborn.pydata.org). Those two libraries are the ones you should be using for the homework.

### Introduction to `seaborn` <a class="anchor" id="section2_1"></a>

`Seaborn` is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics. The library provides a database of useful datasets for educational purposes that can be loaded by typing:

```python
seaborn.load_dataset(name, cache=True, data_home=None, **kws)
```

For information on what these datasets are : https://github.com/mwaskom/seaborn-data

**The plotting functions in `seaborn` can be decided in two categories**

- **'axes-level'** functions, such as `regplot`, `boxplot`, `kdeplot`, `scatterplot`, `distplot` which can connect with the `matplotlib` Axes object and its parameters. You can use that object as you would in `matplotlib`:
```python
f, (ax1, ax2) = plt.subplots(2)
sns.regplot(x, y, ax=ax1)
sns.kdeplot(x, ax=ax2)
ax1 = sns.distplot(x, kde=False, bins=20)
```

- **'figure-level'** functions, such as `lmplot`, `factorplot`, `jointplot`, `relplot`. In this case, `seaborn` organizes the resulting plot which may include several Axes in a meaningful way. That means that the functions need to have total control over the figure, so it isn't possible to plot, say, an `lmplot` onto one that already exists. Calling the function always initializes a figure and sets it up for the specific plot it's drawing. These functions return an object of the type `FacetGrid` with its own methods for operating on the resulting plot.

To set the parameters for figure-level functions:

```python
sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2.5})
```

**The Titanic dataset**

The `titanic.csv` file contains data for 887 passengers on the Titanic. Each row represents one person. The columns describe different attributes about the person including whether they survived, their age, their on-board class, their sex, and the fare they paid.

In [ ]:
titanic = sns.load_dataset("titanic")
titanic.info()

In [ ]:
titanic.columns

### <div class="exercise"><b>Exercise:  Drop the following features</b></div>

'embarked', 'who', 'adult_male', 'embark_town', 'alive', 'alone'

In [ ]:
columns = ["embarked", "who", "adult_male", "embark_town", "alive", "alone"]
titanic = titanic.drop(columns=columns)
titanic

### <div class="exercise"><b>Exercise:  Find for how many passengeres we do not have their deck information.</b></div>

In [ ]:
missing_decks = len(titanic[(pd.isna(titanic["deck"]) == True)])
missing_decks

## Different Plots <a class="anchor" id="chapter3"></a>

### Histograms <a class="anchor" id="section3_1"></a>

**Plotting one variable's distribution (categorical and continous)**
The most convenient way to take a quick look at a univariate distribution in `seaborn` is the `distplot()` function. By default, this will draw a histogram and fit a kernel density estimate (KDE).

A histogram displays a quantitative (numerical) distribution by showing the number (or percentage) of the data values that fall in specified intervals. The intervals are on the x-axis and the number of values falling in each interval, shown as either a number or percentage, are represented by bars drawn above the corresponding intervals.

In [ ]:
# What was the age distribution among passengers in the Titanic?
import seaborn as sns

sns.set(color_codes=True)

f, ax = plt.subplots(1, 1, figsize=(8, 3))
ax = sns.distplot(titanic.age, kde=False, bins=20)

ax.set(xlim=(0, 90))
ax.set_ylabel("counts")

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(8, 3))
ax.hist(titanic.age, bins=20)
ax.set_xlim(0, 90)

### <div class="exercise"><b>Exercise (pandas trick): Count all the infants on board (age less than 3) and all the children ages 5-10.</b></div>

In [ ]:
infants = len(titanic[(titanic.age < 3)])
children = len(titanic[(titanic.age >= 3) & (titanic.age < 10)])
print(f"There were {infants} infants and {children} children on board the Titanic")

**Pandas trick:** We want to creat virtual "bins" for readability and replace ranges of values with categories.

We will do this in an ad hoc way. **It can be done better**. For example in the previous plot we could set:

- `(age<3) = 'infants'`,
- `(3<age<18)='children'`,
- `(18<values<90)='adults'` <BR>

See matplotlib [colors](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html) here.

In [ ]:
# set the colors
cmap = plt.get_cmap("Pastel1")
young = cmap(0.5)
middle = cmap(0.2)
older = cmap(0.8)

# get the object we will change - patches is an array with len: num of bins
fig, ax = plt.subplots()
y_values, bins, patches = ax.hist(titanic.age, 10)

[patches[i].set_facecolor(young) for i in range(0, 1)]  # bin 0
[patches[i].set_facecolor(middle) for i in range(1, 3)]  # bins 1 and 2
[patches[i].set_facecolor(older) for i in range(3, 10)]  # 7 remaining bins

ax.grid(True)
fig.show()

### Kernel Density Estimation <a class="anchor" id="section3_2"></a>
The kernel density estimate can be a useful tool for plotting the shape of a distribution. The **bandwidth (bw)** parameter of the KDE controls how tightly the estimation is fit to the data, much like the bin size in a histogram. It corresponds to the width of the kernels we plotted above. The default behavior tries to guess a good value using a common reference rule, but it may be helpful to try larger or smaller values.

In [ ]:
sns.kdeplot(titanic.age, bw=0.6, label="bw: 0.6", fill=True, color="r")
sns.kdeplot(titanic.age, bw=2, label="bw: 2", fill=True)


### <div class="exercise"><b>Exercise:  Plot the distribution of fare paid by passengers**</b></div>

In [ ]:
# your code here
sns.kdeplot(titanic.fare, bw=0.5, label="bw: 0.5", fill=True)

#### You can mix elements of `matplotlib` such as Axes with `seaborn` elements for a best use of both worlds.

In [ ]:
import seaborn as sns

sns.set(color_codes=True)

x1 = np.random.normal(size=100)
x2 = np.random.normal(size=100)

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# seaborn goes in first subplot
sns.set(font_scale=0.5)
sns.distplot(x1, kde=False, bins=15, ax=ax[0])
sns.distplot(x2, kde=False, bins=15, ax=ax[0])
ax[0].set_title("seaborn Graph Here", fontsize=14)
ax[0].set_xlabel(r"$x$", fontsize=14)
ax[0].set_ylabel(r"$count$", fontsize=14)

# matplotlib goes in second subplot
ax[1].hist(x1, alpha=0.2, bins=15, label=r"$x1$")
ax[1].hist(x2, alpha=0.5, bins=15, label=r"$x2$")
ax[1].set_xlabel(r"$x$", fontsize=14)
ax[1].set_ylabel(r"$count$", fontsize=14)
ax[1].set_title("matplotlib Graph Here", fontsize=14)
ax[1].legend(loc="best", fontsize=14)

#### Introduding the heart disease dataset.


In [ ]:
columns = [
    "age",
    "sex",
    "cp",
    "restbp",
    "chol",
    "fbs",
    "restecg",
    "thalach",
    "exang",
    "oldpeak",
    "slope",
    "ca",
    "thal",
    "num",
]
heart_df = pd.read_csv("data/heart_disease.csv", header=None, names=columns)

heart_df.head()

### Boxplots <a class="anchor" id="section3_3"></a>

#### **One variable:**

In [ ]:
# seaborn
ax = sns.boxplot(x="age", data=titanic)
# ax = sns.boxplot(x=titanic['age']) # another way to write this
ax.set_ylabel(None)
ax.set_xlabel("age", fontsize=14)
ax.set_title("Distribution of age in the Titanic", fontsize=14)

#### **Two variables:**

#### <div class="exercise"><b> Exercise:  Did more young people or older ones get first class tickets on the Titanic?</b></div>

In [ ]:
# two variables seaborn
ax = sns.boxplot(x="class", y="age", data=titanic)

In [ ]:
# two variable boxplot in pandas
titanic.boxplot("age", by="class")

### Scatterplots <a class="anchor" id="section3_4"></a>

#### **Plotting the distribution of two variables**

Also called a bivariate distribution where each observation is shown with  a point with x and y values. You can draw a scatterplot with the `matplotlib plt.scatter` function, or the `seaborn jointplot()` function:

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 5))
sns.scatterplot(x="fare", y="age", data=titanic, ax=ax)

In [ ]:
sns.jointplot(x = "fare", y =  "age", data=titanic, s=40, edgecolor="w", linewidth=1)

You may control the `seaborn` [Figure aesthetics](https://seaborn.pydata.org/tutorial/aesthetics.html).

In [ ]:
# matplotlib
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.scatter(heart_df["age"], heart_df["restbp"], alpha=0.8)
ax.set_xlabel(r"$Age (yrs)$", fontsize=15)
ax.set_ylabel(r"Resting Blood Pressure (mmHg)", fontsize=15)
ax.set_title("Age vs. Resting Blood Pressure", fontsize=14)
plt.show()

#### **Plotting the distribution of three variables**

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 5))
sns.scatterplot(x="fare", y="age", hue="survived", data=titanic, ax=ax)

#### Plotting the distribution of four variables (going too far?)

#### <div class="exercise"><b>Exercise: Plot the distribution of fare paid by passengers according to age, survival and sex.</b></div>

Use `size=` for the fourth variable

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 5))
sns.scatterplot(x="fare", y="age", hue="survived", size="sex", data=titanic, ax=ax)

### Pairplots <a class="anchor" id="section3_5"></a>

In [ ]:
titanic.columns

In [ ]:
to_plot = ["age", "fare", "survived", "deck"]

In [ ]:
df_to_plot = titanic.loc[:, to_plot]
sns.pairplot(df_to_plot)

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(df_to_plot, alpha=0.8, figsize=(10, 10), diagonal="kde")

### Plotting Categorical Variables <a class="anchor" id="section3_6"></a>

In [ ]:
titanic = sns.load_dataset("titanic")
f, ax = plt.subplots(figsize=(7, 3))
ax = sns.countplot(y="deck", data=titanic, color="c")
ax.set_title("Titanic")

In [ ]:
ax = sns.countplot(x="class", data=titanic)
ax.set_title("Titanic")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))  # Create figure object
sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2.5})
ax = sns.countplot(x="deck", data=titanic)

In [ ]:
sns.set(style="ticks", palette="muted")
sns.relplot(x="age", y="deck", col="class", data=titanic)

In [ ]:
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.pairplot(
    data=titanic,
    hue="deck",
    vars=["survived", "pclass", "age", "sibsp", "parch", "fare"],
)

## Introduction to `pandas` plotting. <a class="anchor" id="chapter4"></a>

There is plotting functionality built in `pandas`. Look for it in the `pandas` "encyclopedia", a mere 2883-page pdf from the creator Wes McKinney: [pandas documentation (pdf)](https://pandas.pydata.org/pandas-docs/stable/pandas.pdf)

**Example:** The `value_counts()` Series method and top-level function computes a histogram of a 1D array of values. It can
also be used as a function on regular arrays.

Reminder: DataFrame: “index” (axis=0, default), “columns” (axis=1)

### Line Graph <a class="anchor" id="section4_1"></a>

**Good for time dependency or when a variable evolves**

In [ ]:
df = pd.DataFrame(np.random.randn(1000, 4), columns=["A", "B", "C", "D"])
df.head()

In [ ]:
# cumulative sum adds column values as it goes
df = df.cumsum()
df.head()

In [ ]:
plt.figure()
df.plot()
plt.legend(loc="best")

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range("1/1/2000", periods=1000))
df = pd.DataFrame(np.random.randn(1000, 4), index=ts.index, columns=list("ABCD"))

df = df.cumsum()
plt.figure()
df.plot()

Plotting methods allow for a handful of plot styles other than the default line plot. These methods can be provided as the kind keyword argument to plot(), and include:

- ‘bar’ or ‘barh’ for bar plots
- ‘hist’ for histogram
- ‘box’ for boxplot
- ‘kde’ or ‘density’ for density plots
- ‘area’ for area plots
- ‘scatter’ for scatter plots
- ‘hexbin’ for hexagonal bin plots
- ‘pie’ for pie plots

In addition to these kind s, there are the `DataFrame.hist()`, and `DataFrame.boxplot()` methods, which use a separate interface. `scatter_matrix` in pandas.plotting takes a Series or DataFrame as an argument.

### Bar Plots <a class="anchor" id="section4_2"></a>

In [ ]:
plt.figure()
df.iloc[0].plot(kind="bar")

In [ ]:
df2 = pd.DataFrame(np.random.rand(10, 4), columns=["a", "b", "c", "d"])
df2

In [ ]:
df2.plot.bar()

In [ ]:
# horizontal bar plot
df2.plot.barh(stacked=True)
# Try stacked= False as well.

### Histograms <a class="anchor" id="section4_3"></a>

In [ ]:
df4 = pd.DataFrame(
    {
        "a": np.random.randn(1000) + 1,
        "b": np.random.randn(1000),
        "c": np.random.randn(1000) - 1,
    },
    columns=["a", "b", "c"],
)

plt.figure()
df4.plot.hist(alpha=0.5, stacked=False, bins=60)

### Boxplots <a class="anchor" id="section4_4"></a>

In [ ]:
color = {
    "boxes": "DarkGreen",
    "whiskers": "DarkOrange",
    "medians": "DarkBlue",
    "caps": "Gray",
}

df = pd.DataFrame(np.random.rand(10, 5), columns=["A", "B", "C", "D", "E"])
df.plot.box(color=color)

### Area plots <a class="anchor" id="section4_5"></a>

You can create area plots with `Series.plot.area()` and `DataFrame.plot.area()`. Area plots are stacked
by default. To produce stacked area plot, each column must be either all positive or all negative values.

In [ ]:
df = pd.DataFrame(np.random.rand(10, 4), columns=["a", "b", "c", "d"])
df.plot.area(stacked=True)

In [ ]:
df.plot.area(stacked=False)

### Scatterplot <a class="anchor" id="section4_6"></a>

Scatter plot can be drawn by using the DataFrame.plot.scatter() method. Scatter plot requires numeric
columns for the x and y axes. These can be specified by the x and y keywords.

In [ ]:
ax = df.plot.scatter(x="a", y="b", color="DarkBlue", label="Group 1")
df.plot.scatter(x="c", y="d", color="DarkGreen", label="Group 2", ax=ax)

### `pandas` Tricks

The copy() method on pandas objects copies the underlying data (though not the axis indexes, since they are immutable)
and returns a new object. Note that it is seldom necessary to copy objects. For example, there are only a
handful of ways to alter a DataFrame in-place:

- Inserting, deleting, or modifying a column.
- Assigning to the index or columns attributes.
- For homogeneous data, directly modifying the values via the values attribute or advanced indexing.

To be clear, no pandas method has the side effect of modifying your data; almost every method returns a new object,
leaving the original object untouched. If the data is modified, it is because you did so explicitly


## <div class="exercise"><b>Exercise: 1/2 hour in the Life of a Cardiologist <a class="anchor" id="chapter5"></a></b></div>

Visualize and explore the data. Use `.describe()` to look at your data and also examine if you have any missing values. <BR>
What is the actual number of feature variables after converting categorical variables to dummy ones?<BR><BR>
**List of available variables (includes target variable `num`):**

- **age**: continuous
- **sex**: categorical, 2 values {0: female, 1: male}
- **cp** (chest pain type): categorical, 4 values
    {1: typical angina, 2: atypical angina, 3: non-angina, 4: asymptomatic angina}
- **restbp** (resting blood pressure on admission to hospital): continuous (mmHg)
- **chol (serum cholesterol level)**: continuous (mg/dl)
- **fbs** (fasting blood sugar): categorical, 2 values {0: <= 120 mg/dl, 1: > 120 mg/dl}
- **restecg** (resting electrocardiography): categorical, 3 values
    {0: normal, 1: ST-T wave abnormality, 2: left ventricular hypertrophy}
- **thalach** (maximum heart rate achieved): continuous
- **exang** (exercise induced angina): categorical, 2 values {0: no, 1: yes}
- **oldpeak** (ST depression induced by exercise relative to rest): continuous
- **slope** (slope of peak exercise ST segment): categorical, 3 values
    {1: upsloping, 2: flat, 3: downsloping}
- **ca** (number of major vessels colored by fluoroscopy): discrete (0,1,2,3)
- **thal**: categorical, 3 values {3: normal, 6: fixed defect, 7: reversible defect}
- **num** (diagnosis of heart disease): categorical, 5 values
    {0: less than 50% narrowing in any major vessel,
    1-4: more than 50% narrowing in 1-4 vessels}

In [ ]:
# load the dataset
heart_df = pd.read_csv("data/heart_disease.csv", header=None, names=columns)
heart_df.head()

**We will answer the following question using plots**

01. At what ages do people seek cardiological exams?
02. Do men seek help more than women?
03. Examine the variables. How do they relate to one another?
04. (Variation on 02): What % of men and women seek cardio exams?
05. Does resting blood pressure increase with age?

**Pandas trick: `.replace`** The response variable (num) is categorical with 5 values, but we don't have enough data to predict all the categories. <BR> Therefore we'll replace `num` with `hd` (heart disease): **categorical, 2 values {0: no, 1: yes}**. <BR>
Use the code below (take a minute to understand how it works, it's very useful!):

In [ ]:
# Replace response variable values with a binary response (1: heart disease(hd) or 0: not)
heart_df["num"].replace(to_replace=[1, 2, 3, 4], value=1, inplace=True)

# Rename column for clarity
heart_df = heart_df.rename(columns={"num": "hd"})
heart_df.head()

In [ ]:
# look at the features
heart_df.info()

In [ ]:
heart_df.describe()

your code here
01. what ages do people seek cardiological exams?

In [ ]:
# your code here
# 02. do men seek help more than women?

# %load solutions/q02.py

In [ ]:
# your code here
# 03. Examine the variables. How do they relate to one another?

# %load solutions/q03.py

In [ ]:
# your code here
# 04. What percentage of men and women seek cardio exams?

# %load solutions/q04.py

In [ ]:
# your code here
# 05. Does resting blood pressure increase with age?

# %load solutions/q05.py

### <div class="exercise"><b>Exercise: Find the hidden pattern</b></div>

Read the following file into a `pandas` Dataframe: 'data/mystery.csv' and plot it.  How does it look? You should see a beautiful pattern. If not, think of ways to fix the issue.

In [ ]:
mystery = pd.read_csv("data/mystery.csv", sep=" ", header=None)
mystery.head()

your code here